In [ ]:
!pip uninstall scikit-learn

Found existing installation: scikit-learn 1.6.0
Uninstalling scikit-learn-1.6.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/scikit_learn-1.6.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.10/dist-packages/sklearn/*
Proceed (Y/n)? y
  Successfully uninstalled scikit-learn-1.6.0


In [ ]:
# Imports and setup
!python -m spacy download en_core_web_lg
!pip install scikit-learn==1.5.2
!pip install gensim plotly==4.14.3

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, f1_score
from gensim.models import Word2Vec
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
import spacy
import re
from collections import defaultdict
import os


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 51.0 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support, classification_report
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from collections import defaultdict, Counter
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
nlp = spacy.load('en_core_web_lg')

# Paths
USER_STORIES = [
    ('Camperplus', 'camperplus.txt'),
    ('Fish&Chips', 'Fish_Chips.txt'),
    ('Grocery', 'grocery.txt'),
    ('Planningpoker', 'planningpoker.txt'),
    ('Recycling', 'recycling.txt'),
    ('School', 'school.txt'),
    ('Sports', 'sports.txt'),
    ('Supermarket', 'supermarket.txt'),
    ('Ticket', 'Ticket.txt')
]
GOLD_STANDARD_CLASSES = 'Gold standard - Classes.xlsx'
GOLD_STANDARD_ASSOCIATIONS = 'Gold standard - Associations.xlsx'
CLASS_OUTPUT = 'class_features.csv'
ASSOCIATION_OUTPUT = 'association_features.csv'

# Helper function to read files with fallback encoding
def read_file_with_encoding(file_path):
    """Read a file with fallback encoding."""
    try:
        with open(file_path, encoding='utf-8') as f:
            return f.readlines()
    except UnicodeDecodeError:
        with open(file_path, encoding='cp1252') as f:
            return f.readlines()

# Extract features for class identification
def extract_features_for_classes(user_story_path, gold_standard_classes_path, class_column):
    """Extract features for class identification."""
    if not os.path.exists(user_story_path):
        print(f"Error: User story file not found -> {user_story_path}")
        return

    if not os.path.exists(gold_standard_classes_path):
        print(f"Error: Gold standard classes file not found -> {gold_standard_classes_path}")
        return

    # Read user stories
    lines = read_file_with_encoding(user_story_path)

    # Read gold standard
    gold_standard = pd.read_excel(gold_standard_classes_path, engine='openpyxl')
    if class_column not in gold_standard.columns:
        print(f"Error: Column '{class_column}' not found in Gold Standard Classes.")
        return

    class_labels = set(stemmer.stem(label.lower()) for label in gold_standard[class_column].dropna())

    entities = defaultdict(lambda: {
        'count': 0, 'noun': 0, 'subject': 0, 'compound': 0, 'gerund': 0,
        'user_role': 0, 'action': 0, 'benefit': 0, 'part_of_frequent_ngram': 0, 'target': 0
    })

    global_ngrams = Counter()

    for line in lines:
        doc = nlp(line)
        words = [token.text.lower() for token in doc if token.text.lower() not in stop_words]

        # Collect n-grams globally across all lines
        ngrams = [' '.join(words[i:i+n]) for n in range(2, 5) for i in range(len(words) - n + 1)]
        global_ngrams.update(ngrams)

        for token in doc:
            word = stemmer.stem(token.text.lower())
            if len(word) <= 2 or word in stop_words:
                continue

            pos = token.tag_
            dep = token.dep_
            entities[word]['count'] += 1
            if word in class_labels:
                entities[word]['target'] = 1
            if 'NN' in pos:
                entities[word]['noun'] += 1
            if dep == 'nsubj':
                entities[word]['subject'] += 1
            if dep == 'compound':
                entities[word]['compound'] += 1
            if pos == 'VBG':
                entities[word]['gerund'] += 1

        # Extract role, action, benefit
        if line.lower().startswith("as a "):
            parts = line.split(",")
            if len(parts) >= 3:
                role = parts[0][5:].strip()
                action = parts[1][8:].strip()
                benefit = parts[2][8:].strip()
                for token in nlp(role):
                    entities[token.text.lower()]['user_role'] += 1
                for token in nlp(action):
                    entities[token.text.lower()]['action'] += 1
                for token in nlp(benefit):
                    entities[token.text.lower()]['benefit'] += 1

    # Update entities with global n-gram frequencies
    for ngram, freq in global_ngrams.items():
        if freq >= 2:
            entities[ngram]['part_of_frequent_ngram'] = 1

    data = pd.DataFrame.from_dict(entities, orient='index')
    data.index.name = 'phrase'
    data.reset_index(inplace=True)
    data.to_csv(CLASS_OUTPUT.replace('features.csv', f'{class_column}_features.csv'), index=False)
    print(f"Class features for {class_column} saved.")

# Extract features for association identification
def extract_features_for_associations(user_story_path, gold_standard_assoc_path, project_name):
    """Extract features for association identification."""
    if not os.path.exists(user_story_path):
        print(f"Error: User story file not found -> {user_story_path}")
        return

    if not os.path.exists(gold_standard_assoc_path):
        print(f"Error: Gold standard associations file not found -> {gold_standard_assoc_path}")
        return

    # Load gold standard associations
    gold_standard = pd.read_excel(gold_standard_assoc_path, engine='openpyxl')
    if project_name not in gold_standard.columns or f"Unnamed: {gold_standard.columns.get_loc(project_name)+1}" not in gold_standard.columns:
        print(f"Error: Columns for project '{project_name}' not found in Gold Standard Associations.")
        return

    associations = set(
        zip(
            gold_standard[project_name].dropna().str.lower(),
            gold_standard[f"Unnamed: {gold_standard.columns.get_loc(project_name)+1}"].dropna().str.lower()
        )
    )

    classes = {assoc[0] for assoc in associations} | {assoc[1] for assoc in associations}

    # Initialize features
    features = []
    for class_a in classes:
        for class_b in classes:
            if class_a == class_b:
                continue

            # External Knowledge: Similarity
            similarity = cosine_similarity([nlp(class_a).vector], [nlp(class_b).vector])[0][0]

            # Internal Knowledge: Co-occurrence
            cooccurrence = sum(1 for line in read_file_with_encoding(user_story_path)
                               if class_a in line and class_b in line)

            # Internal Knowledge: Individual counts
            count_a = sum(1 for line in read_file_with_encoding(user_story_path) if class_a in line)
            count_b = sum(1 for line in read_file_with_encoding(user_story_path) if class_b in line)

            # Requirements-Type Knowledge: Role, Action, Benefit
            role_overlap = 0
            action_overlap = 0
            benefit_overlap = 0
            for line in read_file_with_encoding(user_story_path):
                if line.lower().startswith("as a "):
                    parts = line.split(",")
                    if len(parts) >= 3:
                        role = parts[0][5:].strip()
                        action = parts[1][8:].strip()
                        benefit = parts[2][8:].strip()
                        if class_a in role and class_b in role:
                            role_overlap += 1
                        if class_a in action and class_b in action:
                            action_overlap += 1
                        if class_a in benefit and class_b in benefit:
                            benefit_overlap += 1

            # Add feature dictionary
            features.append({
                'class_a': class_a,
                'class_b': class_b,
                'similarity': similarity,
                'cooccurrence': cooccurrence,
                'count_a': count_a,
                'count_b': count_b,
                'role_overlap': role_overlap,
                'action_overlap': action_overlap,
                'benefit_overlap': benefit_overlap,
                'target': int((class_a, class_b) in associations or (class_b, class_a) in associations)
            })

    # Save features to a CSV
    feature_df = pd.DataFrame(features)
    feature_df.to_csv(ASSOCIATION_OUTPUT.replace('features.csv', f'{project_name}_features.csv'), index=False)
    print(f"Association features for {project_name} saved.")

# Main Processing Loop
for project_name, user_story_file in USER_STORIES:
    print(f"\nProcessing project: {project_name}")
    extract_features_for_classes(user_story_file, GOLD_STANDARD_CLASSES, project_name)
    extract_features_for_associations(user_story_file, GOLD_STANDARD_ASSOCIATIONS, project_name)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Processing project: Camperplus
Class features for Camperplus saved.
Association features for Camperplus saved.

Processing project: Fish&Chips
Class features for Fish&Chips saved.
Association features for Fish&Chips saved.

Processing project: Grocery
Class features for Grocery saved.
Association features for Grocery saved.

Processing project: Planningpoker
Class features for Planningpoker saved.
Association features for Planningpoker saved.

Processing project: Recycling
Class features for Recycling saved.
Association features for Recycling saved.

Processing project: School
Class features for School saved.
Association features for School saved.

Processing project: Sports
Class features for Sports saved.
Association features for Sports saved.

Processing project: Supermarket
Class features for Supermarket saved.
Association features for Supermarket saved.

Processing project: Ticket
Class features for Ticket saved.
Association features for Ticket saved.


In [ ]:
!pip install scikeras

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, Dropout, Input
from scikeras.wrappers import KerasClassifier
from sklearn.exceptions import ConvergenceWarning
import warnings
import os

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

def balance_data(X, y):
    ros = RandomOverSampler(random_state=21)
    X_res, y_res = ros.fit_resample(X, y)
    return X_res, y_res

def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv1D(64, kernel_size=min(3, input_shape[0]), activation='relu'))
    if input_shape[0] > 1:
        model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(LSTM(64, return_sequences=False, dropout=0.5, recurrent_dropout=0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def tune_hyperparameters(model, param_grid, X_train, y_train):
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='f1', cv=3, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_

def train_models(features, target_col, task_name):
    features = features.select_dtypes(include=[np.number])
    if target_col not in features.columns:
        print(f"Error: Target column '{target_col}' not found in features for {task_name}.")
        return

    X = features.drop(columns=[target_col])
    y = features[target_col]

    if y.nunique() == 1:
        print(f"Skipping {task_name}: Only one class present. Model training not possible.")
        return

    X_balanced, y_balanced = balance_data(X, y)

    models = {
    'Logistic Regression': (
        LogisticRegression(max_iter=1000, class_weight='balanced', solver='lbfgs'),
        {
            'C': [0.01, 0.1, 1, 10]
        }
    ),
    'Random Forest': (
        RandomForestClassifier(class_weight='balanced', random_state=42),
        {
            'n_estimators': [100, 200],
            'max_depth': [None, 10, 20]
        }
    ),
    'Gradient Boosting': (
        GradientBoostingClassifier(random_state=30),
        {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1]
        }
    ),
    'SVM': (
        SVC(probability=True, class_weight='balanced', random_state=20),
        {
            'C': [0.1, 1, 10],
            'kernel': ['linear', 'rbf']
        }
    ),
    'MLP': (
        MLPClassifier(max_iter=1000, random_state=15, early_stopping=True, validation_fraction=0.2),
        {
            'hidden_layer_sizes': [(100,), (50, 50)],
            'alpha': [0.0001, 0.001]
        }
    ),
    'Naive Bayes': (
        GaussianNB(),
        {}
    )
}


    base_learners = [(name, model[0]) for name, model in models.items() if name != 'MLP']
    stacking_clf = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression(max_iter=1000))
    models['Stacking Classifier'] = (stacking_clf, {})

    cnn_model = KerasClassifier(model=create_cnn_model, input_shape=(X_balanced.shape[1], 1), epochs=10, batch_size=32, verbose=0)
    lstm_model = KerasClassifier(model=create_lstm_model, input_shape=(X_balanced.shape[1], 1), epochs=10, batch_size=32, verbose=0)
    models['CNN'] = (cnn_model, {})
    models['LSTM'] = (lstm_model, {})

    results = []
    skf = StratifiedKFold(n_splits=8, shuffle=True)

    for name, (model, param_grid) in models.items():
        f0_5_scores, f1_scores, f2_scores = [], [], []
        precision_scores, recall_scores = [], []

        for train_idx, test_idx in skf.split(X_balanced, y_balanced):
            X_train, X_test = X_balanced.iloc[train_idx], X_balanced.iloc[test_idx]
            y_train, y_test = y_balanced.iloc[train_idx], y_balanced.iloc[test_idx]

            scaler = MinMaxScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            if name in ['CNN', 'LSTM']:
                X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
                X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

            if param_grid:
                model = tune_hyperparameters(model, param_grid, X_train_scaled, y_train)

            pipeline = Pipeline([
                ('model', model)
            ])
            pipeline.fit(X_train_scaled, y_train)
            y_pred = pipeline.predict(X_test_scaled)

            precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary', zero_division=0)
            f0_5 = (1 + 0.5 ** 2) * (precision * recall) / ((0.5 ** 2 * precision) + recall) if (precision + recall) > 0 else 0
            f2 = (1 + 2 ** 2) * (precision * recall) / ((2 ** 2 * precision) + recall) if (precision + recall) > 0 else 0

            f0_5_scores.append(f0_5)
            f1_scores.append(f1)
            f2_scores.append(f2)
            precision_scores.append(precision)
            recall_scores.append(recall)

        results.append({
            'Model': name,
            'Precision': np.mean(precision_scores),
            'Recall': np.mean(recall_scores),
            'F0.5': np.mean(f0_5_scores),
            'F1': np.mean(f1_scores),
            'F2': np.mean(f2_scores)
        })

    results_df = pd.DataFrame(results)
    print(f"\n{task_name} Results:")
    print(results_df)

if __name__ == "__main__":
    project_files = [
        ("Camperplus", "class_Camperplus_features.csv", "association_Camperplus_features.csv"),
        ("Fish&Chips", "class_Fish&Chips_features.csv", "association_Fish&Chips_features.csv"),
        ("Grocery", "class_Grocery_features.csv", "association_Grocery_features.csv"),
        ("Planningpoker", "class_Planningpoker_features.csv", "association_Planningpoker_features.csv"),
        ("Recycling", "class_Recycling_features.csv", "association_Recycling_features.csv"),
        ("School", "class_School_features.csv", "association_School_features.csv"),
        ("Sports", "class_Sports_features.csv", "association_Sports_features.csv"),
        ("Supermarket", "class_Supermarket_features.csv", "association_Supermarket_features.csv"),
        ("Ticket", "class_Ticket_features.csv", "association_Ticket_features.csv")
    ]

    for project_name, class_file, assoc_file in project_files:
        print(f"\nProcessing project: {project_name}")

        if os.path.exists(class_file):
            class_features = pd.read_csv(class_file)
            print(f"\nClass Identification Models for {project_name}:")
            train_models(class_features, 'target', f"{project_name} Class Identification")
        else:
            print(f"Skipping: Class features file not found -> {class_file}")

        if os.path.exists(assoc_file):
            assoc_features = pd.read_csv(assoc_file)
            print(f"\nAssociation Identification Models for {project_name}:")
            train_models(assoc_features, 'target', f"{project_name} Association Identification")
        else:
            print(f"Skipping: Association features file not found -> {assoc_file}")



Processing project: Camperplus

Class Identification Models for Camperplus:

Camperplus Class Identification Results:
                 Model  Precision    Recall      F0.5        F1        F2
0  Logistic Regression   0.887400  1.000000  0.907583  0.939886  0.974924
1        Random Forest   0.955860  1.000000  0.964326  0.977356  0.990798
2    Gradient Boosting   0.948301  1.000000  0.958134  0.973342  0.989132
3                  SVM   0.856064  1.000000  0.881339  0.922274  0.967336
4                  MLP   0.841497  0.680897  0.788783  0.732070  0.695676
5          Naive Bayes   0.606603  1.000000  0.658281  0.754842  0.884880
6  Stacking Classifier   0.953209  1.000000  0.961993  0.975682  0.990034
7                  CNN   0.865361  0.596328  0.785639  0.696706  0.631212
8                 LSTM   0.784272  0.820763  0.772941  0.774846  0.796007

Association Identification Models for Camperplus:

Camperplus Association Identification Results:
                 Model  Precision    Recal